In [56]:
from qiskit import *
from qiskit_aer import *
from qiskit.primitives import *
from qiskit.circuit.library import *
from qiskit.visualization import *
from math import *

### Generate a list of all marked states less than K

In [57]:
def generate_marked_states(k, intList, num_qubits):
    """Generate binary strings for integers less than k and present in intList."""
    return [f"{i:0{num_qubits}b}" for i in range(k) if i in intList], num_qubits

### Create the oracle used in the Grover's algorithm

In [58]:
def grover_oracle(k, intList, num_qubits):
    """Create a Grover oracle that marks states less than k and present in intList."""
    marked_states, _ = generate_marked_states(k, intList, num_qubits)
    qc = QuantumCircuit(num_qubits)

    for target in marked_states:
        zero_inds = [i for i, bit in enumerate(reversed(target)) if bit == '0']
        qc.x(zero_inds)

        if num_qubits > 1:
            qc.h(num_qubits - 1)
            qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)
            qc.h(num_qubits - 1)
        else:
            qc.z(0)

        qc.x(zero_inds)
    orcale = qc.to_gate()
    return orcale

### Create the diffusion operator used in the Grover's algorithm used to amplify the amplitude of the marked states

In [59]:
def diffusion_operator(num_qubits):
    """Constructs the diffusion operator for Grover's algorithm."""
    qc = QuantumCircuit(num_qubits)

    qc.h(range(num_qubits))
    qc.x(range(num_qubits))

    # MCZ
    if num_qubits >= 2:
        qc.h(num_qubits-1)
        qc.mcx(list(range(num_qubits-1)), num_qubits-1)
        qc.h(num_qubits-1)
    else:
        qc.z(0)

    qc.x(range(num_qubits))
    qc.h(range(num_qubits))

    diffusion = qc.to_gate()
    return diffusion

### Algorithm that uses Grover's algorithm to find the marked states

In [60]:
def less_than_k(k, intList):
    """Returns a list of integers less than k and present in intList."""
    if not intList:
        raise ValueError("intList must not be empty.")
    num_qubits = ceil(log2(max(intList) + 1))
    marked_states, _ = generate_marked_states(k, intList, num_qubits)

    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))

    # Oracle
    oracle = grover_oracle(k, intList, num_qubits)
    qc.append(oracle, range(num_qubits))

    # Diffusion Operator
    diffusion_op = diffusion_operator(num_qubits)
    qc.append(diffusion_op, range(num_qubits))

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    # Execute the Circuit
    backend = Aer.get_backend('qasm_simulator')
    result = backend.run(transpile(qc, backend)).result()
    counts = result.get_counts(qc)
    topSamples = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:len(marked_states)]
    topKeys = [int(key, 2) for key, value in topSamples]

    return topKeys

# Example

In [61]:
A = less_than_k(7, [4,9,11,14,1,13,6,15])
print(A)

[4, 1, 6]
